# Poetry Generation (Tensorflow, Keras, LSTM)

In [1]:
import string
import requests
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
filenames = ['dataset/adele.txt', 'dataset/lady-gaga.txt',
             'dataset/kanye-west.txt', 'dataset/eminem.txt']

In [3]:
with open('dataset/MERGED.txt', 'w', encoding='utf-8') as outfile:
    for fname in filenames:
        with open(fname, 'r', encoding='utf-8') as infile:
            outfile.write(infile.read() + '\n')

In [4]:
with open('dataset/MERGED.txt', 'r', encoding='utf-8') as file:
    data = file.read().splitlines()

In [5]:
len(data)

19212

In [ ]:
' '.join(data)

## Building LSTM Model

In [7]:
token = Tokenizer()
token.fit_on_texts(data)
# token.word_counts, token.word_index

In [8]:
encoded_text = token.texts_to_sequences(data)
encoded_text[:3]

[[276, 26, 106, 3698], [153, 7, 64, 218, 2, 140], [28, 11, 993, 1730]]

In [9]:
x = ['Token Text']
token.texts_to_sequences(x)

[[4467, 1965]]

In [10]:
vocab_size = len(token.word_counts) + 1

## Prepare Training Data

In [11]:
datalist = []
for d in encoded_text:
    if len(d) > 1:
        for i in range(2, len(d)):
            datalist.append(d[:i])

In [12]:
datalist[4]

[153, 7, 64, 218]

## Padding

In [13]:
max_length = 50
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
sequences[:4]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0, 276,  26],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0, 276,  26, 106],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0, 153,   7],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
     

In [14]:
sequences.shape

(127241, 50)

In [15]:
X = sequences[:, :-1]
X[:4]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0, 276],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0, 276,  26],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0, 153],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   

In [16]:
y = sequences[:, -1]
vocab_size

10571

In [17]:
y = to_categorical(y, num_classes=vocab_size)
y.shape, y

((127241, 10571),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [18]:
seq_length = X.shape[1]
seq_length

49

## LSTM Model Training

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.build(input_shape=(None, seq_length))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          528550    
                                                                 
 lstm (LSTM)                 (None, None, 100)         60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 10571)             1067671   
                                                                 
Total params: 1,747,121
Trainable params: 1,747,121
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=32, epochs=100)

Epoch 1/100
3977/3977 [==============================] - 58s 13ms/step - loss: 6.2630 - accuracy: 0.0478
Epoch 2/100
3977/3977 [==============================] - 52s 13ms/step - loss: 5.7889 - accuracy: 0.0679
Epoch 3/100
3977/3977 [==============================] - 51s 13ms/step - loss: 5.5532 - accuracy: 0.0892
Epoch 4/100
3977/3977 [==============================] - 50s 13ms/step - loss: 5.3284 - accuracy: 0.1092
Epoch 5/100
3977/3977 [==============================] - 50s 13ms/step - loss: 5.1363 - accuracy: 0.1225
Epoch 6/100
3977/3977 [==============================] - 50s 13ms/step - loss: 4.9773 - accuracy: 0.1337
Epoch 7/100
3977/3977 [==============================] - 50s 13ms/step - loss: 4.8417 - accuracy: 0.1453
Epoch 8/100
3977/3977 [==============================] - 50s 13ms/step - loss: 4.7209 - accuracy: 0.1553
Epoch 9/100
3977/3977 [==============================] - 52s 13ms/step - loss: 4.6138 - accuracy: 0.1645
Epoch 10/100
3977/3977 [==============================]

## Poetry Generation

In [ ]:
poetry_length = 10

def generate_poetry(seed_text, n_lines):
    for i in range(n_lines):
        text = []
        for _ in range(poetry_length):
            encoded = token.texts_to_sequences([seed_text])[0]
            encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')

            y_pred = np.argmax(model.predict(encoded, verbose=0), axis=-1)

            predicted_word = ''
            for word, index in token.word_index.items():
                if index == y_pred:
                    predicted_word = word
                    break
            
            seed_text = seed_text + ' ' + predicted_word
            text.append(predicted_word)

        seed_text = text[-1]
        text = ' '.join(text)
        print(text)

In [2]:
# generate_poetry('sex', 2)

## Model Save & Load

In [30]:
from tensorflow.keras.models import load_model

In [32]:
model.save('model\model_poetry_1.h5')

In [ ]:
# model = load_model('model\model_poetry.h5')
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X, y, batch_size=16, epochs=20)